## **This file contains the strategy for combining static and dynamic features using Autoencoder**


In [ ]:
#import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
import tensorflow.keras as keras
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
#static data file
data = pd.read_csv("./soil_data.csv")
data

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1005,31.870670,-85.405456,146,0.0158,0.1868,0.5441,0.2424,0.0106,0.0003,...,72.578804,1.828159,74.406960,3,2,1,1,1,1,1
2,1003,30.659218,-87.746067,52,0.0746,0.4370,0.4415,0.0469,0.0000,0.0000,...,59.843639,2.996914,62.840553,3,2,1,2,1,1,1
3,1007,33.015893,-87.127148,93,0.0144,0.1617,0.3714,0.3493,0.0898,0.0134,...,1.916593,0.008330,1.924924,3,2,1,1,1,1,1
4,1009,33.977448,-86.567246,198,0.0050,0.0872,0.2799,0.3576,0.1477,0.1037,...,1.891909,0.027488,1.919397,3,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,56037,41.660339,-108.875676,2085,0.0016,0.0281,0.1763,0.3554,0.2121,0.2097,...,0.000000,0.000000,0.000000,1,1,3,1,1,1,3
3105,56039,44.049321,-110.588102,2564,0.0003,0.0026,0.0166,0.0722,0.1489,0.5005,...,2.922309,0.000000,2.922309,1,1,1,1,1,1,1
3106,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
3107,56041,41.284726,-110.558947,2327,0.0050,0.2009,0.4063,0.1858,0.0964,0.1031,...,1.013702,10.755590,11.769293,1,1,2,1,1,1,2


In [ ]:
#dynamic data file
train_data = pd.read_csv("./train_timeseries.csv")
train_data

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,...,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81,NaN
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,...,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41,NaN
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66,NaN
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58,1.0
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19300675,56043,2016-12-27,0.16,82.88,1.63,-7.97,-13.49,-12.81,-1.39,-13.60,...,-9.41,5.90,7.63,3.61,4.02,8.58,10.39,5.92,4.47,0.0
19300676,56043,2016-12-28,0.02,83.33,1.41,-8.71,-14.10,-13.84,-2.49,-13.56,...,-10.55,6.50,11.43,4.11,7.32,9.92,14.49,7.26,7.22,NaN
19300677,56043,2016-12-29,0.00,83.75,1.59,-7.96,-13.30,-13.03,0.42,-14.51,...,-10.29,4.29,6.24,2.03,4.22,6.56,10.07,3.20,6.87,NaN
19300678,56043,2016-12-30,1.22,82.49,2.63,-2.94,-7.40,-7.33,3.76,-6.86,...,-4.14,4.98,7.34,1.99,5.35,7.28,10.12,3.24,6.89,NaN


In [ ]:
train_data = train_data[train_data['date'] <= '2012-12-31']

In [ ]:
#merge both files
result = pd.merge(train_data, data, on='fips', how='inner')

In [ ]:
result

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759887,56043,2012-12-27,1.33,82.23,1.76,-8.43,-11.79,-11.67,-4.52,-10.35,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759888,56043,2012-12-28,0.08,82.80,1.53,-8.00,-13.29,-13.07,-1.95,-11.53,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759889,56043,2012-12-29,0.01,82.92,1.29,-10.03,-15.05,-14.77,-3.07,-15.59,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759890,56043,2012-12-30,0.02,82.83,1.37,-9.48,-14.42,-14.16,-2.66,-13.37,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
result =  result.drop(['score'], axis=1) #drop main target column

In [ ]:
#split into train, validation, test
train = result[(result['date'] >= '2000-01-01') & (result['date'] <= '2008-12-31')]
validation = result[(result['date'] >= '2009-01-01') & (result['date'] <= '2010-12-31')]
test = result[(result['date'] >= '2011-01-01') & (result['date'] <= '2012-12-31')]

In [ ]:
train

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,2000-01-01,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,2000-01-02,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,2000-01-03,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,2000-01-04,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,2000-01-05,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14758426,56043,2008-12-27,0.70,82.31,1.47,-10.39,-14.50,-14.23,-2.84,-16.79,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14758427,56043,2008-12-28,0.28,82.80,2.66,-4.52,-7.41,-7.37,2.14,-9.60,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14758428,56043,2008-12-29,0.36,82.77,3.35,-1.00,-4.38,-4.36,4.18,-5.80,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14758429,56043,2008-12-30,1.08,83.28,1.71,-4.75,-12.42,-12.11,2.01,-10.46,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
validation

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
3288,1001,2009-01-01,0.01,100.69,3.66,3.63,-0.75,-0.72,11.42,-3.64,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3289,1001,2009-01-02,2.71,100.15,7.29,9.71,8.96,8.96,13.03,5.97,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3290,1001,2009-01-03,9.09,100.12,10.66,15.36,14.91,14.91,19.45,11.91,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3291,1001,2009-01-04,6.31,100.37,10.56,15.55,14.88,14.88,19.34,13.36,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3292,1001,2009-01-05,18.56,100.17,11.01,16.24,15.44,15.44,19.49,12.75,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759156,56043,2010-12-27,0.00,83.02,1.95,-4.33,-10.64,-10.47,2.10,-8.03,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759157,56043,2010-12-28,0.03,82.35,2.08,-5.43,-10.13,-10.01,1.07,-10.15,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759158,56043,2010-12-29,1.26,80.57,2.93,-1.48,-6.29,-6.24,3.36,-5.36,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759159,56043,2010-12-30,3.67,81.43,1.15,-12.76,-16.88,-16.48,-6.34,-17.29,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
test

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
4018,1001,2011-01-01,40.22,100.10,8.99,13.17,12.08,12.08,17.51,7.61,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4019,1001,2011-01-02,0.02,100.85,3.34,3.48,-2.08,-2.01,8.69,-3.74,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4020,1001,2011-01-03,0.00,100.93,2.60,2.19,-5.02,-4.91,10.40,-6.08,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4021,1001,2011-01-04,0.05,100.53,3.67,6.28,-0.59,-0.55,13.98,0.18,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4022,1001,2011-01-05,4.24,99.86,5.88,6.97,5.78,5.79,10.42,2.27,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14759887,56043,2012-12-27,1.33,82.23,1.76,-8.43,-11.79,-11.67,-4.52,-10.35,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759888,56043,2012-12-28,0.08,82.80,1.53,-8.00,-13.29,-13.07,-1.95,-11.53,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759889,56043,2012-12-29,0.01,82.92,1.29,-10.03,-15.05,-14.77,-3.07,-15.59,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
14759890,56043,2012-12-30,0.02,82.83,1.37,-9.48,-14.42,-14.16,-2.66,-13.37,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
#split date into year, month, date in train set
train['date'] = pd.to_datetime(train['date'])

train['Year'] = train['date'].dt.year
train['Month'] = train['date'].dt.month
train['Day'] = train['date'].dt.day

train = train.drop(columns=['date'])
train

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,...,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,Year,Month,Day
0,1001,0.22,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,...,1,1,1,1,1,1,2,2000,1,1
1,1001,0.20,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,...,1,1,1,1,1,1,2,2000,1,2
2,1001,3.65,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,...,1,1,1,1,1,1,2,2000,1,3
3,1001,15.95,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,...,1,1,1,1,1,1,2,2000,1,4
4,1001,0.00,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,...,1,1,1,1,1,1,2,2000,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219099,56043,0.70,82.31,1.47,-10.39,-14.50,-14.23,-2.84,-16.79,13.95,...,1,1,1,1,1,1,1,2008,12,27
10219100,56043,0.28,82.80,2.66,-4.52,-7.41,-7.37,2.14,-9.60,11.74,...,1,1,1,1,1,1,1,2008,12,28
10219101,56043,0.36,82.77,3.35,-1.00,-4.38,-4.36,4.18,-5.80,9.98,...,1,1,1,1,1,1,1,2008,12,29
10219102,56043,1.08,83.28,1.71,-4.75,-12.42,-12.11,2.01,-10.46,12.46,...,1,1,1,1,1,1,1,2008,12,30


In [ ]:
#split date into year, month, date in validation set
validation['date'] = pd.to_datetime(validation['date'])

validation['Year'] = validation['date'].dt.year
validation['Month'] = validation['date'].dt.month
validation['Day'] = validation['date'].dt.day

validation = validation.drop(columns=['date'])
validation

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,...,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,Year,Month,Day
0,1001,0.01,100.69,3.66,3.63,-0.75,-0.72,11.42,-3.64,15.06,...,1,1,1,1,1,1,2,2009,1,1
1,1001,2.71,100.15,7.29,9.71,8.96,8.96,13.03,5.97,7.06,...,1,1,1,1,1,1,2,2009,1,2
2,1001,9.09,100.12,10.66,15.36,14.91,14.91,19.45,11.91,7.54,...,1,1,1,1,1,1,2,2009,1,3
3,1001,6.31,100.37,10.56,15.55,14.88,14.88,19.34,13.36,5.98,...,1,1,1,1,1,1,2,2009,1,4
4,1001,18.56,100.17,11.01,16.24,15.44,15.44,19.49,12.75,6.74,...,1,1,1,1,1,1,2,2009,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268835,56043,0.00,83.02,1.95,-4.33,-10.64,-10.47,2.10,-8.03,10.12,...,1,1,1,1,1,1,1,2010,12,27
2268836,56043,0.03,82.35,2.08,-5.43,-10.13,-10.01,1.07,-10.15,11.22,...,1,1,1,1,1,1,1,2010,12,28
2268837,56043,1.26,80.57,2.93,-1.48,-6.29,-6.24,3.36,-5.36,8.72,...,1,1,1,1,1,1,1,2010,12,29
2268838,56043,3.67,81.43,1.15,-12.76,-16.88,-16.48,-6.34,-17.29,10.94,...,1,1,1,1,1,1,1,2010,12,30


In [ ]:
#split date into year, month, date in test set
test['date'] = pd.to_datetime(test['date'])

test['Year'] = test['date'].dt.year
test['Month'] = test['date'].dt.month
test['Day'] = test['date'].dt.day

test = test.drop(columns=['date'])
test

,fips,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,...,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7,Year,Month,Day
0,1001,40.22,100.10,8.99,13.17,12.08,12.08,17.51,7.61,9.90,...,1,1,1,1,1,1,2,2011,1,1
1,1001,0.02,100.85,3.34,3.48,-2.08,-2.01,8.69,-3.74,12.43,...,1,1,1,1,1,1,2,2011,1,2
2,1001,0.00,100.93,2.60,2.19,-5.02,-4.91,10.40,-6.08,16.48,...,1,1,1,1,1,1,2,2011,1,3
3,1001,0.05,100.53,3.67,6.28,-0.59,-0.55,13.98,0.18,13.80,...,1,1,1,1,1,1,2,2011,1,4
4,1001,4.24,99.86,5.88,6.97,5.78,5.79,10.42,2.27,8.15,...,1,1,1,1,1,1,2,2011,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271943,56043,1.33,82.23,1.76,-8.43,-11.79,-11.67,-4.52,-10.35,5.83,...,1,1,1,1,1,1,1,2012,12,27
2271944,56043,0.08,82.80,1.53,-8.00,-13.29,-13.07,-1.95,-11.53,9.58,...,1,1,1,1,1,1,1,2012,12,28
2271945,56043,0.01,82.92,1.29,-10.03,-15.05,-14.77,-3.07,-15.59,12.52,...,1,1,1,1,1,1,1,2012,12,29
2271946,56043,0.02,82.83,1.37,-9.48,-14.42,-14.16,-2.66,-13.37,10.71,...,1,1,1,1,1,1,1,2012,12,30


In [ ]:
print(train.columns)

Index(['fips', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET', 'T2M_MAX',
       'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
       'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE', 'lat',
       'lon', 'elevation', 'slope1', 'slope2', 'slope3', 'slope4', 'slope5',
       'slope6', 'slope7', 'slope8', 'aspectN', 'aspectE', 'aspectS',
       'aspectW', 'aspectUnknown', 'WAT_LAND', 'NVG_LAND', 'URB_LAND',
       'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND', 'CULTIR_LAND', 'CULT_LAND',
       'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6', 'SQ7', 'Year', 'Month',
       'Day'],
      dtype='object')


In [ ]:
#check target columns
dynamic_columns = ['Year', 'Month', 'Day', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET','T2M_MAX','T2M_MIN','T2M_RANGE','TS', 'WS10M', 'WS10M_MAX','WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE']  #dynamic features

In [ ]:
dynamic_columns

['Year',
 'Month',
 'Day',
 'PS',
 'QV2M',
 'T2M',
 'T2MDEW',
 'T2MWET',
 'T2M_MAX',
 'T2M_MIN',
 'T2M_RANGE',
 'TS',
 'WS10M',
 'WS10M_MAX',
 'WS10M_MIN',
 'WS10M_RANGE',
 'WS50M',
 'WS50M_MAX',
 'WS50M_MIN',
 'WS50M_RANGE']

In [ ]:
y_train = train['PRECTOT'] #target column "PRECTOT"
y_train

0            0.22
1            0.20
2            3.65
3           15.95
4            0.00
            ...  
10219099     0.70
10219100     0.28
10219101     0.36
10219102     1.08
10219103     0.54
Name: PRECTOT, Length: 10219104, dtype: float64

In [ ]:
y_valid = validation['PRECTOT']
y_valid

0           0.01
1           2.71
2           9.09
3           6.31
4          18.56
           ...  
2268835     0.00
2268836     0.03
2268837     1.26
2268838     3.67
2268839     0.77
Name: PRECTOT, Length: 2268840, dtype: float64

In [ ]:
y_test = test['PRECTOT']
y_test

0          40.22
1           0.02
2           0.00
3           0.05
4           4.24
           ...  
2271943     1.33
2271944     0.08
2271945     0.01
2271946     0.02
2271947     0.01
Name: PRECTOT, Length: 2271948, dtype: float64

In [ ]:
#drop target column
train = train.drop(columns=['PRECTOT'])
validation = validation.drop(columns=['PRECTOT'])
test = test.drop(columns=['PRECTOT'])

In [ ]:
dynamic_train = train[dynamic_columns]#seperate dynamic features
dynamic_train

,Year,Month,Day,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,2000,1,1,100.51,9.65,14.74,13.51,13.51,20.96,11.46,9.50,14.65,2.20,2.94,1.49,1.46,4.85,6.04,3.23,2.81
1,2000,1,2,100.55,10.42,16.69,14.71,14.71,22.80,12.61,10.18,16.60,2.52,3.43,1.83,1.60,5.33,6.13,3.72,2.41
2,2000,1,3,100.15,11.76,18.49,16.52,16.52,22.73,15.32,7.41,18.41,4.03,5.33,2.66,2.67,7.53,9.52,5.87,3.66
3,2000,1,4,100.29,6.42,11.40,6.09,6.10,18.09,2.16,15.92,11.31,3.84,5.67,2.08,3.59,6.73,9.31,3.74,5.58
4,2000,1,5,101.15,2.95,3.86,-3.29,-3.20,10.82,-2.66,13.48,2.65,1.60,2.50,0.52,1.98,2.94,4.85,0.65,4.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219099,2008,12,27,82.31,1.47,-10.39,-14.50,-14.23,-2.84,-16.79,13.95,-12.92,4.23,5.29,2.87,2.43,6.19,7.66,4.25,3.41
10219100,2008,12,28,82.80,2.66,-4.52,-7.41,-7.37,2.14,-9.60,11.74,-6.26,5.65,7.75,4.19,3.55,8.59,9.63,7.34,2.30
10219101,2008,12,29,82.77,3.35,-1.00,-4.38,-4.36,4.18,-5.80,9.98,-2.90,7.62,12.24,4.11,8.13,11.32,17.66,6.97,10.69
10219102,2008,12,30,83.28,1.71,-4.75,-12.42,-12.11,2.01,-10.46,12.46,-6.65,6.06,12.57,2.33,10.24,8.87,17.68,3.82,13.86


In [ ]:
dynamic_valid = validation[dynamic_columns] #seperate dynamic features
dynamic_valid

,Year,Month,Day,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,2009,1,1,100.69,3.66,3.63,-0.75,-0.72,11.42,-3.64,15.06,3.69,2.57,3.82,1.82,2.00,5.10,6.62,4.17,2.45
1,2009,1,2,100.15,7.29,9.71,8.96,8.96,13.03,5.97,7.06,9.31,1.51,2.53,0.24,2.29,2.85,5.03,0.38,4.65
2,2009,1,3,100.12,10.66,15.36,14.91,14.91,19.45,11.91,7.54,15.24,2.17,2.83,1.82,1.01,4.82,5.91,3.50,2.41
3,2009,1,4,100.37,10.56,15.55,14.88,14.88,19.34,13.36,5.98,15.39,1.70,1.96,1.12,0.85,3.63,4.77,2.03,2.74
4,2009,1,5,100.17,11.01,16.24,15.44,15.44,19.49,12.75,6.74,16.11,1.98,4.01,0.54,3.47,4.06,7.64,1.10,6.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268835,2010,12,27,83.02,1.95,-4.33,-10.64,-10.47,2.10,-8.03,10.12,-6.26,4.08,6.90,2.39,4.51,6.39,8.52,3.75,4.77
2268836,2010,12,28,82.35,2.08,-5.43,-10.13,-10.01,1.07,-10.15,11.22,-6.70,4.55,6.31,2.68,3.63,6.89,9.48,4.39,5.09
2268837,2010,12,29,80.57,2.93,-1.48,-6.29,-6.24,3.36,-5.36,8.72,-2.66,5.22,8.11,1.26,6.85,7.25,11.04,2.30,8.74
2268838,2010,12,30,81.43,1.15,-12.76,-16.88,-16.48,-6.34,-17.29,10.94,-11.67,4.79,5.88,3.68,2.20,5.70,6.95,4.13,2.81


In [ ]:
dynamic_test = test[dynamic_columns] #seperate dynamic features
dynamic_test

,Year,Month,Day,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE
0,2011,1,1,100.10,8.99,13.17,12.08,12.08,17.51,7.61,9.90,13.04,3.44,4.77,1.99,2.78,6.22,8.63,3.25,5.38
1,2011,1,2,100.85,3.34,3.48,-2.08,-2.01,8.69,-3.74,12.43,3.45,2.93,4.45,1.90,2.55,5.47,6.77,4.59,2.18
2,2011,1,3,100.93,2.60,2.19,-5.02,-4.91,10.40,-6.08,16.48,0.54,1.23,2.28,0.19,2.09,2.32,4.72,0.23,4.49
3,2011,1,4,100.53,3.67,6.28,-0.59,-0.55,13.98,0.18,13.80,4.16,1.64,2.30,0.72,1.58,2.51,3.74,1.22,2.52
4,2011,1,5,99.86,5.88,6.97,5.78,5.79,10.42,2.27,8.15,6.39,1.82,2.84,0.13,2.70,3.49,5.89,0.21,5.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271943,2012,12,27,82.23,1.76,-8.43,-11.79,-11.67,-4.52,-10.35,5.83,-9.06,3.48,4.97,1.72,3.25,4.51,5.77,2.93,2.84
2271944,2012,12,28,82.80,1.53,-8.00,-13.29,-13.07,-1.95,-11.53,9.58,-10.04,2.96,5.31,1.86,3.45,4.41,6.46,2.97,3.49
2271945,2012,12,29,82.92,1.29,-10.03,-15.05,-14.77,-3.07,-15.59,12.52,-11.64,2.99,4.06,1.63,2.43,4.43,5.41,2.67,2.74
2271946,2012,12,30,82.83,1.37,-9.48,-14.42,-14.16,-2.66,-13.37,10.71,-10.99,1.75,2.86,0.80,2.06,2.52,4.82,0.83,3.99


In [ ]:
static_train = train.drop(columns=dynamic_columns) #seperate static features
static_train

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219099,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
10219100,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
10219101,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
10219102,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
static_valid = validation.drop(columns=dynamic_columns) #seperate static features
static_valid

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268835,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2268836,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2268837,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2268838,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
static_test = test.drop(columns=dynamic_columns)#seperate static features
static_test

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,CULTRF_LAND,CULTIR_LAND,CULT_LAND,SQ1,SQ2,SQ3,SQ4,SQ5,SQ6,SQ7
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,56.293411,1.014811,57.308224,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271943,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2271944,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2271945,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1
2271946,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.000000,0.000000,0.000000,1,1,1,1,1,1,1


In [ ]:
# Define the dimensions of the autoencoder
dynamic_input_dim = dynamic_train.shape[1]
dynamic_encoding_dim = 8

In [ ]:
# Define the autoencoder model for dynamic features
dynamic_autoencoder = Sequential()
dynamic_autoencoder.add(Dense(dynamic_encoding_dim, activation='relu', input_shape=(dynamic_input_dim,)))
dynamic_autoencoder.add(Dense(dynamic_input_dim, activation='relu'))

In [ ]:
# Compile and train the autoencoder for dynamic features
dynamic_autoencoder.compile(optimizer='adam', loss='mean_absolute_error')
dynamic_autoencoder.fit(dynamic_train, dynamic_train, epochs=10, batch_size=32)

Epoch 1/10
319347/319347 [==============================] - 543s 2ms/step - loss: 5.7822
Epoch 2/10
319347/319347 [==============================] - 532s 2ms/step - loss: 5.6591
Epoch 3/10
319347/319347 [==============================] - 532s 2ms/step - loss: 5.6562
Epoch 4/10
319347/319347 [==============================] - 526s 2ms/step - loss: 5.6548
Epoch 5/10
319347/319347 [==============================] - 525s 2ms/step - loss: 5.5495
Epoch 6/10
319347/319347 [==============================] - 532s 2ms/step - loss: 5.4284
Epoch 7/10
319347/319347 [==============================] - 534s 2ms/step - loss: 5.1458
Epoch 8/10
319347/319347 [==============================] - 526s 2ms/step - loss: 5.1444
Epoch 9/10
319347/319347 [==============================] - 522s 2ms/step - loss: 5.1417
Epoch 10/10
319347/319347 [==============================] - 522s 2ms/step - loss: 5.1388


In [ ]:
dynamic_train_autoencoder = dynamic_autoencoder.predict(dynamic_train)
dynamic_validation_autoencoder = dynamic_autoencoder.predict(dynamic_valid)
dynamic_test_autoencoder = dynamic_autoencoder.predict(dynamic_test)

70999/70999 [==============================] - 63s 889us/step


In [ ]:
dynamic_validation_autoencoder

In [ ]:
dynamic_test_autoencoder

In [ ]:
dynamic_train_df = pd.DataFrame(dynamic_train_autoencoder)
dynamic_valid_df = pd.DataFrame(dynamic_validation_autoencoder)
dynamic_test_df = pd.DataFrame(dynamic_test_autoencoder)

In [ ]:
dynamic_train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2000.174683,0.0,0.0,100.509735,10.254373,14.775581,13.447180,13.342274,0.0,0.0,0.0,14.521126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.184565
1,2000.184570,0.0,0.0,100.549797,10.835546,16.675438,14.660075,14.554793,0.0,0.0,0.0,16.497709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.192181
2,2000.162476,0.0,0.0,100.140724,11.783827,18.464008,16.486208,16.378775,0.0,0.0,0.0,18.322374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.108471
3,2000.227539,0.0,0.0,100.311729,6.533755,11.376548,6.205066,6.113139,0.0,0.0,0.0,11.236746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.846046
4,2000.222534,0.0,0.0,101.153282,1.926449,3.218920,0.000000,0.000000,0.0,0.0,0.0,2.944411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.320693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219099,2008.199219,0.0,0.0,82.264557,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.451045
10219100,2008.154907,0.0,0.0,82.785217,1.666696,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.133389
10219101,2008.197510,0.0,0.0,82.762589,3.050765,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.065495
10219102,2008.245117,0.0,0.0,83.239700,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.723710


In [ ]:
dynamic_valid_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2009.186401,0.0,0.0,100.702248,3.270509,3.793186,0.000000,0.000000,0.0,0.0,0.0,3.435192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.071623
1,2009.167358,0.0,0.0,100.162460,8.107534,9.654498,8.941561,8.840359,0.0,0.0,0.0,9.249921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.346355
2,2009.157104,0.0,0.0,100.118927,11.077809,15.389938,14.867801,14.759913,0.0,0.0,0.0,15.103436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.068015
3,2009.157837,0.0,0.0,100.367691,11.028543,15.549017,14.824337,14.716836,0.0,0.0,0.0,15.277278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.087821
4,2009.181763,0.0,0.0,100.167633,11.311935,16.268372,15.368044,15.260138,0.0,0.0,0.0,16.020477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.079616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268835,2010.165894,0.0,0.0,82.987556,0.061589,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.538682
2268836,2010.168335,0.0,0.0,82.326515,0.373390,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.421394
2268837,2010.168823,0.0,0.0,80.557564,2.292599,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.301843
2268838,2010.207886,0.0,0.0,81.383995,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.725154


In [ ]:
dynamic_test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2011.183472,0.0,0.0,100.111565,9.651062,13.226196,12.027929,11.923978,0.0,0.0,0.0,12.924228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.257389
1,2011.170410,0.0,0.0,100.861252,2.623819,3.556155,0.000000,0.000000,0.0,0.0,0.0,3.242168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.217978
2,2011.218872,0.0,0.0,100.929390,1.204716,1.321236,0.000000,0.000000,0.0,0.0,0.0,0.985612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.391076
3,2011.194092,0.0,0.0,100.536331,3.348536,5.087414,0.000000,0.000000,0.0,0.0,0.0,4.812413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.167752
4,2011.181519,0.0,0.0,99.876945,6.579000,6.840798,5.827139,5.729600,0.0,0.0,0.0,6.386332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.514565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271943,2012.126831,0.0,0.0,82.198372,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.403797
2271944,2012.156128,0.0,0.0,82.756462,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.544276
2271945,2012.179688,0.0,0.0,82.875374,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.603718
2271946,2012.172729,0.0,0.0,82.784782,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.583138


In [ ]:
#Combine static and dynamic features
train_combined  = pd.concat([static_train, dynamic_train_df], axis=1)
validation_combined  = pd.concat([static_valid, dynamic_valid_df], axis=1)
test_combined = pd.concat([static_test, dynamic_test_df], axis=1)

In [ ]:
train_combined

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,10,11,12,13,14,15,16,17,18,19
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,14.521126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.184565
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,16.497709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.192181
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,18.322374,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.108471
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,11.236746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.846046
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,2.944411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.320693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219099,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.451045
10219100,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.133389
10219101,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.065495
10219102,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.723710


In [ ]:
validation_combined

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,10,11,12,13,14,15,16,17,18,19
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,3.435192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.071623
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,9.249921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.346355
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,15.103436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.068015
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,15.277278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.087821
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,16.020477,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.079616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2268835,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.538682
2268836,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.421394
2268837,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.301843
2268838,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.725154


In [ ]:
test_combined

,fips,lat,lon,elevation,slope1,slope2,slope3,slope4,slope5,slope6,...,10,11,12,13,14,15,16,17,18,19
0,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,12.924228,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.257389
1,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,3.242168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.217978
2,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,0.985612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.391076
3,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,4.812413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.167752
4,1001,32.536382,-86.644490,63,0.0419,0.2788,0.2984,0.2497,0.1142,0.0170,...,0.0,6.386332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.514565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271943,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.403797
2271944,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.544276
2271945,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.603718
2271946,56043,43.878831,-107.669052,1417,0.0034,0.0470,0.2331,0.4099,0.2064,0.0999,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.583138


In [ ]:
#save combined file
train_combined.to_csv('C:/Users/anon/Desktop/tensorflow_test/train_combined.csv', index=False)
validation_combined.to_csv('C:/Users/anon/Desktop/tensorflow_test/validation_combined.csv', index=False)
test_combined.to_csv('C:/Users/anon/Desktop/tensorflow_test/test_combined.csv' , index=False)

**The futher implementation part of this in file encoder_next_file...**

